In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from IPython.core.display import HTML

# Combining all data

## Crash

In [2]:
#Column 22 (Rpt_Block_Num) is almost all int except for when they put 111 1/2 or 111 BLK
rpt_block_num_cnvtr = lambda x: x.split(' ')[0] if len(x.split(' ')) > 1 and x.split(' ')[1] in ['1/2', 'BLK'] else x
rpt_sec_hwy_sfx_dict = {'W': 22, 'R': 17, 'E': 5, 'S': 18, 'N': 14, 'M': 13, 'A': 1}
rpt_sec_hwy_sfx_cnvtr = lambda x: rpt_sec_hwy_sfx_dict[x] if type(x) == str and len(x) > 0 else x
crash_10_cnvtr = {22: rpt_block_num_cnvtr}
crash_12_cnvtr = {22: rpt_block_num_cnvtr, 35: rpt_sec_hwy_sfx_cnvtr, 20: rpt_sec_hwy_sfx_cnvtr}
dtypes = {'Rpt_Sec_Hwy_Num': str, 'Rpt_Sec_Block_Num': str, 'Street_Nbr': str, \
          'Rpt_CrossingNumber': str, 'Rpt_CrossingNumber': str, 'CrossingNumber': str, 'RRCo': str}

In [3]:
# crash_10 skipped rows have data split incorrectly in csv file starting at col 33
# crash_15, 16, 17 skipped rows have Rpt_Hwy_Sfx values entered incorrectly 
#     (entered actual sfx rather than data dict code)
crash_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_crash_20100101-20101231_TRAVIS.csv', \
                       converters=crash_10_cnvtr, skiprows=[2495, 3258, 3580, 8252, 9523], dtype=dtypes, \
                       index_col='Crash_ID')
crash_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_crash_20110101-20111231_TRAVIS.csv', \
                      converters=crash_10_cnvtr, dtype=dtypes, index_col='Crash_ID')
crash_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_crash_20120101-20121231_TRAVIS.csv', \
                      converters=crash_12_cnvtr, dtype=dtypes, index_col='Crash_ID')
crash_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_crash_20130101-20131231_TRAVIS.csv', \
                      dtype=dtypes, index_col='Crash_ID')
crash_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_crash_20140101-20141231_TRAVIS.csv', \
                       dtype=dtypes, index_col='Crash_ID')
crash_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_crash_20150101-20151231_TRAVIS.csv', \
                       converters=crash_12_cnvtr, dtype=dtypes, index_col='Crash_ID', \
                       skiprows=[5177, 5446, 6653, 9839, 9841, 14381, 19176])
crash_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_crash_20160101-20161231_TRAVIS.csv', \
                       converters=crash_12_cnvtr, dtype=dtypes, skiprows=[2432, 3794, 3795, 8077, 8188, 20173], \
                       index_col='Crash_ID')
crash_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_crash_20170101-20170829_TRAVIS.csv', \
                      converters=crash_12_cnvtr, dtype=dtypes, skiprows=[4480], index_col='Crash_ID')

/home/michael/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
crash = pd.concat([crash_10, crash_11, crash_12, crash_13, crash_14, crash_15, crash_16, crash_17])
crash['Crash_Timestamp'] = crash['Crash_Date'] + ' ' + crash['Crash_Time']
crash['Crash_Timestamp'] = pd.to_datetime(crash['Crash_Timestamp'])
crash = crash.drop(['Crash_Date', 'Crash_Time'], axis=1)

### Checking no new duplicates from concat

In [5]:
# crash[crash.Crash_ID.duplicated()]

## Damages

In [6]:
damages_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_damages_20100101-20101231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_damages_20110101-20111231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_damages_20120101-20121231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_damages_20130101-20131231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_damages_20140101-20141231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_damages_20150101-20151231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_damages_20160101-20161231_TRAVIS.csv', \
                         index_col='Crash_ID')
damages_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_damages_20170101-20170829_TRAVIS.csv', \
                         index_col='Crash_ID')

In [7]:
damages = pd.concat([damages_10, damages_11, damages_12, damages_13, damages_14, damages_15, damages_16, damages_17])

### Checking no new duplicates from concat

In [8]:
# damages['dup'] = damages['Crash_ID'].map(str) + damages['Damaged_Property'].map(str)
# damages = damages.sort_values(by='dup')
# duplicates shown exist in csv. must be 'multiple counts of damages' or something like that
# damages[damages['dup'].duplicated(keep=False)]


## Unit

In [9]:
cmv_carrier_zip_cnvtr = lambda x: x.replace('-', '') if '-' in str(x) else x
unit_10_cnvtr = {31: cmv_carrier_zip_cnvtr}
dtypes = {80: str}

In [10]:
# Unit_16 skipped rows had column values shifted in the csv
unit_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_unit_20100101-20101231_TRAVIS.csv', \
                     converters=unit_10_cnvtr, index_col='Crash_ID')
unit_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_unit_20110101-20111231_TRAVIS.csv', \
                      index_col='Crash_ID')
unit_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_unit_20120101-20121231_TRAVIS.csv', \
                      index_col='Crash_ID')
unit_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_unit_20130101-20131231_TRAVIS.csv', \
                      index_col='Crash_ID')
unit_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_unit_20140101-20141231_TRAVIS.csv', \
                      index_col='Crash_ID')
unit_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_unit_20150101-20151231_TRAVIS.csv', \
                      index_col='Crash_ID')
unit_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_unit_20160101-20161231_TRAVIS.csv', \
                     skiprows=[43205], dtype=dtypes, index_col='Crash_ID')
unit_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_unit_20170101-20170829_TRAVIS.csv', \
                      index_col='Crash_ID')

/home/michael/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
unit = pd.concat([unit_10, unit_11, unit_12, unit_13, unit_14, unit_15, unit_16, unit_17])
unit = unit.drop(['Incap_Injry_Cnt', 'Nonincap_Injry_Cnt', 'Poss_Injry_Cnt', 'Non_Injry_Cnt', \
                        'Unkn_Injry_Cnt', 'Tot_Injry_Cnt', 'Death_Cnt'], axis=1)

### Checking no new duplicates from concat

In [12]:
# unit['dup'] = unit['Crash_ID'].map(str) + unit['Unit_Nbr'].map(str)
# unit = unit.sort_values(by='dup')
# unit[unit['dup'].duplicated(keep=False)]


## PrimaryPerson

In [13]:
primaryperson_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_primaryperson_20100101-20101231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_primaryperson_20110101-20111231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_primaryperson_20120101-20121231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_primaryperson_20130101-20131231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_primaryperson_20140101-20141231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_primaryperson_20150101-20151231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_primaryperson_20160101-20161231_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])
primaryperson_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_primaryperson_20170101-20170829_TRAVIS.csv', \
                               index_col=['Crash_ID', 'Unit_Nbr'])

In [14]:
primaryperson = pd.concat([primaryperson_10, primaryperson_11, primaryperson_12, primaryperson_13, primaryperson_14, primaryperson_15, primaryperson_16, primaryperson_17])
primaryperson = primaryperson.drop(['Incap_Injry_Cnt', 'Nonincap_Injry_Cnt', 'Poss_Injry_Cnt', 'Non_Injry_Cnt', \
                        'Unkn_Injry_Cnt', 'Tot_Injry_Cnt', 'Death_Cnt'], axis=1)

### Checking no new duplicates from concat

In [15]:
# primaryperson['dup'] = primaryperson['Crash_ID'].map(str) + primaryperson['Unit_Nbr'].map(str) + primaryperson['Prsn_Nbr'].map(str)
# primaryperson = primaryperson.sort_values(by='dup')
# primaryperson[primaryperson['dup'].duplicated(keep=False)]


## Person

In [16]:
person_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_person_20100101-20101231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_person_20110101-20111231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_person_20120101-20121231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_person_20130101-20131231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_person_20140101-20141231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_person_20150101-20151231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_person_20160101-20161231_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])
person_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_person_20170101-20170829_TRAVIS.csv', \
                        index_col=['Crash_ID', 'Unit_Nbr'])

In [17]:
person = pd.concat([person_10, person_11, person_12, person_13, person_14, person_15, person_16, person_17])
person = person.drop(['Incap_Injry_Cnt', 'Nonincap_Injry_Cnt', 'Poss_Injry_Cnt', 'Non_Injry_Cnt', \
                        'Unkn_Injry_Cnt', 'Tot_Injry_Cnt', 'Death_Cnt'], axis=1)

### Checking no new duplicates from concat

In [18]:
# person['dup'] = person['Crash_ID'].map(str) + person['Unit_Nbr'].map(str) + person['Prsn_Nbr'].map(str)
# person = person.sort_values(by='dup')
# person[person['dup'].duplicated(keep=False)]


## Endorsements

In [19]:
endorsements_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_endorsements_20100101-20101231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_endorsements_20110101-20111231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_endorsements_20120101-20121231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_endorsements_20130101-20131231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_endorsements_20140101-20141231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_endorsements_20150101-20151231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_endorsements_20160101-20161231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
endorsements_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_endorsements_20170101-20170829_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])

In [20]:
endorsements = pd.concat([endorsements_10, endorsements_11, endorsements_12, endorsements_13, endorsements_14, endorsements_15, endorsements_16, endorsements_17])
endorsements['Prsn_Nbr'] = 1.0
endorsements = endorsements.reset_index().set_index(['Crash_ID', 'Unit_Nbr', 'Prsn_Nbr'])

### Checking no new duplicates from concat

In [21]:
# endorsements['dup'] = endorsements['Crash_ID'].map(str) + endorsements['Unit_Nbr'].map(str) + endorsements['Drvr_Lic_Endors_ID'].map(str)
# endorsements = endorsements.sort_values(by='dup')
# endorsements[endorsements['dup'].duplicated(keep=False)]


## Restrictions

In [22]:
restrictions_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_restrictions_20100101-20101231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_restrictions_20110101-20111231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_restrictions_20120101-20121231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_restrictions_20130101-20131231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_restrictions_20140101-20141231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_restrictions_20150101-20151231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_restrictions_20160101-20161231_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])
restrictions_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_restrictions_20170101-20170829_TRAVIS.csv', \
                              index_col=['Crash_ID', 'Unit_Nbr'])

In [23]:
restrictions = pd.concat([restrictions_10, restrictions_11, restrictions_12, restrictions_13, restrictions_14, restrictions_15, restrictions_16, restrictions_17])
restrictions['Prsn_Nbr'] = 1.0
restrictions = restrictions.reset_index().set_index(['Crash_ID', 'Unit_Nbr', 'Prsn_Nbr'])

### Checking no new duplicates from concat

In [24]:
# restrictions['dup'] = restrictions['Crash_ID'].map(str) + restrictions['Unit_Nbr'].map(str) + restrictions['Drvr_Lic_Restric_ID'].map(str)
# restrictions = restrictions.sort_values(by='dup')
# restrictions[restrictions['dup'].duplicated(keep=False)]

## Charges

In [25]:
charges_10 = pd.read_csv('data/2010/extract_public_2010_20170913160331_charges_20100101-20101231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_11 = pd.read_csv('data/2011/extract_public_2010_20170913151637_charges_20110101-20111231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_12 = pd.read_csv('data/2012/extract_public_2010_20170913151619_charges_20120101-20121231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_13 = pd.read_csv('data/2013/extract_public_2010_20170913151600_charges_20130101-20131231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_14 = pd.read_csv('data/2014/extract_public_2010_20170913151542_charges_20140101-20141231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_15 = pd.read_csv('data/2015/extract_public_2015_20170829153840_charges_20150101-20151231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_16 = pd.read_csv('data/2016/extract_public_2015_20170829153815_charges_20160101-20161231_TRAVIS.csv', \
                         index_col='Crash_ID')
charges_17 = pd.read_csv('data/2017/extract_public_2015_20170829132341_charges_20170101-20170829_TRAVIS.csv', \
                         index_col='Crash_ID')

In [26]:
charges = pd.concat([charges_10, charges_11, charges_12, charges_13, charges_14, charges_15, charges_16, charges_17])
charges = charges.reset_index().set_index(['Crash_ID', 'Unit_Nbr', 'Prsn_Nbr'])

### Checking no new duplicates from concat

In [27]:
# charges['dup'] = charges.Crash_ID.map(str) + charges.Unit_Nbr.map(str) + charges.Prsn_Nbr.map(str) + charges.Charge_Cat_ID.map(str) + charges.Charge + charges.Citation_Nbr.map(str)
# charges = charges.sort_values(by='dup')
# charges[charges.dup.duplicated(keep=False)]

## Joining each section of data

In [28]:
joined = crash.join(damages, how='left')
joined = joined.join(unit, how='left')
joined = joined.reset_index().set_index(['Crash_ID', 'Unit_Nbr'])

people = pd.concat([person, primaryperson])
people = people.sort_index()
joined = joined.join(people, how='left')
joined = joined.reset_index().set_index(['Crash_ID', 'Unit_Nbr', 'Prsn_Nbr'])
joined = joined.sort_index()
joined = joined.join(endorsements, how='left')
joined = joined.join(restrictions, how='left')
joined = joined.join(charges, how='left')

## Featurizing data